In [1]:
import pandas as pd
import matplotlib as mpl
import plotly.express as px
import scipy.stats as stats
import numpy as np

mpl.rc('font', family = 'AppleGothic')

In [2]:
df1 = pd.read_csv('01_Data.csv')

# 기계학습의 종류

1. 지도 학습(Supervised Learning) : 목표변수(Y)와 설명변수(X)의 관계를 학습해서, 새로운 X가 들어올 때, Y를 예측/분류
    - 회귀 (Regression - Y 연속형) : 특정 값을 최대한 가깝게 예측하는 것이 목적
    - 분류 (Classification - Y 범주형) : 특정 항목인지 아닌지를 정확하게 분류하는 것이 목적(평가 지표들이 많다!)
    - Ex) 고객 해약 예측 (정상 / 해악) / 스팸메세지 분류기 (스팸/정상) / 주가 예측 (회사의 정보 -> 주가) ...
    
2. 비지도학습(Unsupervised Learning) : 설명변수(X) 데이터 간의 수학적 거리/ 유사성 / 상관성 등을 이용하여, 비슷한 유형의 데이터를 묶거나 연관있는 데이터를 찾거나, 데이터의 차원을 줄이는 등의 학습 기법
    - 군집분석 (Clustering) : 비슷한 위치 또는 가가운 위치의 (특성이 비슷한) 데이터를 묶어주는 학습 기법
    - 차원축소 : 특정 데이터의 항목 수 (Columns)를 줄이는 기법
    - 연관분석 : 특정 데이터의 유사한 항목을 찾아주는 기법
    Ex) 장바구니 분석 / 추천 시스템 ...
    
    
- 준지도 학습 (Semi Supervised learning) : 비지도학습 + 지도학습 결합

3. 강화학습(Reinforcement Learning) : 컴퓨터 시뮬레이션을 통해 사용자가 설정한 환경에 대해, 적절한 보상이 주어지는 방향으로 학습을 수행
    - 데이터가 없이도 학습이 가능
    - 게임 AI (알파고)

**지도 학습 절차**

1. 데이터 핸들링 (데이터를 병합 / 파생 변수 생성 / 데이터 선택 / 이상치 제거 /...)
2. 학습의 목표변수(Y)와 설명변수(X)를 설정
    - 유의 사항 : 사용되는 X 설명변수는 새로 들어올 데이터에 대한 값으로 설정
3. 학습 데이터(Train Set)와 검증 데이터(Test Set)를 분할
    - 검증 데이터(Test Set)는 절대로 학습에 참여하지 않는다!
    - (교차검증 기법에서 Validation Set은 학습에 참여함 <-> Test Set과 별개!)
4. 학습 수행
    - 특성공학 (Feature Engineering) : 학습에 맞게 데이터를 처리
        - 스케일링 / 인코딩 / 교차검증 / 변수선택법 ...
    - 학습
5. 학습 된 모델의 성능을 평가
    - 학습 능력
    - 일반화 능력
6. 적용(새로운 데이터 입력 / 파일형태로 저장)

- 고객들의 정보를 이용하여, 해약 여부를 판별하는 모델을 생성

In [3]:
# 1. 데이터 핸들링
df1['State'].unique()

array(['계약확정', '기간만료', '해약확정', '해약진행중'], dtype=object)

In [4]:
df1['해약여부'] = df1['State'].replace({'계약확정':0, '기간만료':1,
                                   '해약확정':1, '해약진행중':1})
df1['해약여부'].value_counts()

해약여부
0    50620
1      681
Name: count, dtype: int64

In [5]:
df2 = df1.dropna() # 결측값제거

In [6]:
#2. 목표변수 Y / 설명변수 X 선택
Y = df2['해약여부']
X = df2[['Age', 'Credit_Rank', 'Amount_Month', 'Term']]

In [7]:
# 3. 학습 데이터와 검증 데이터를 분할
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,
                                                   random_state = 1234)

In [9]:
#4. 학습을 수행
from sklearn.tree import DecisionTreeClassifier

In [10]:
model = DecisionTreeClassifier()
model.fit(X_train, Y_train)

DecisionTreeClassifier()

In [11]:
# 5. 평가 수행 (학습능력 / 일반화능력)
# 기존에 사용한 X 데이터를 이용해 예측값을 계산
Y_train_pred = model.predict(X_train)
Y_test_pred = model.predict(X_test)

In [12]:
from sklearn.metrics import accuracy_score #정확도 평가 지표

In [13]:
#학습 능력 평가
accuracy_score(Y_train, Y_train_pred)

0.9876660652780056

In [14]:
#일반화 능력 평가
accuracy_score(Y_test, Y_test_pred)

0.9850423145050187

In [15]:
# 6. 적용
x1 = input('고객 연령을 입력하시오 : ')
x2 = input('고객 신용 등급을 입력하시오 : ')
x3 = input('계약할 월 랜탈비용을 입력하시오 : ')
x4 = input('계약할 기간을 입력하시오 : ')

고객 연령을 입력하시오 : 28
고객 신용 등급을 입력하시오 : A
계약할 월 랜탈비용을 입력하시오 : 30
계약할 기간을 입력하시오 : 2


In [16]:
input_data = pd.DataFrame([[x1,x2,x3,x4]], columns=X.columns)

In [17]:
model.predict(input_data) # 해약 : 1 / 정상 : 0

ValueError: could not convert string to float: 'A'

In [18]:
# 생성된 모델을 파일 형태로 저장
import pickle # 파이썬에 있는 객체나 변수를 파일형태로 저장

In [ ]:
pickle.dump(model, open('model.sav', 'wb'))

# 모델평가

- 분류모델에서의 평가
- **정확도 (Accuracy)** = 예측 결과가 동일한 데이터의 수 / 전체 예측 데이터의 수
    - 클래스(나누고자 하는 항목)의 비율이 균형을 이룰 때 주로 사용
    - 모든 클래스가 동등한 중요도를 가지며, 비율도 같을 때
    - 단순 이진분류에서 데이터의 비율이 깨져있는 경우 (Imbalanced Data) 모델의 성능이 잘못 측정됨
- **오차 행렬 (Confusion Matrix)**
    - 정상 (Negative)을 정상으로 해약(Positive)을 해약으로 분류하는 데이터의 수를 표로 정리
    
- **정밀도 (Precision)** : True Positive / (False Positive + True Positive)
    - (정확하게 해약으로 분류한 수) / (예측을 해약으로 분류한 수)
    - 예측 성능을 더욱 정밀하게 측정하기 위해 사용되는 지표
    - False Positive의 개수를 낮추는데에 초점 / ex) 스팸메시지 분류
    
- **재현율 (Recall)** : True Positive / (False Negative + True Positive)
    - (정확하게 해약으로 분류한 수) / (실제 해약인 데이터 수)
    - Sensitivity (민감도) / TPR(True Positive Rate)
    - 실제 문제가 있는 데이터를 문제가 없다고 잘못 판단할 때 발생하는 이슈를 나타내는 지표
    - False Negative를 낮추는 데 초점 / ex) 암진단, 불량여부, 해약여부 ...
    
- 두 수치가 모두 좋아야 한다.

In [19]:
from sklearn.metrics import precision_score, recall_score

In [20]:
# 0 / 1 의 이진 분류에서 1값을 Positive로 뒀을 때, 정밀도
print('학습 데이터의 Precision : ', precision_score(Y_train, Y_train_pred))
print('검증 데이터의 Precision : ', precision_score(Y_test, Y_test_pred))

학습 데이터의 Precision :  0.9574468085106383
검증 데이터의 Precision :  0.08823529411764706


In [21]:
print('학습 데이터의 Recall : ', recall_score(Y_train, Y_train_pred))
print('검증 데이터의 Recall : ', recall_score(Y_test, Y_test_pred))
#비율이 깨진 데이터에서 Recall을 중점으로 보는 실무적 데이터의 경우
#Recall 값이 Precision에 비해 현저히 낮게 나오는 경향

학습 데이터의 Recall :  0.10739856801909307
검증 데이터의 Recall :  0.024193548387096774


In [22]:
#분류모델 평가 지표를 모두 확인
from sklearn.metrics import classification_report

In [23]:
print(classification_report(Y_train, Y_train_pred)) #학습

              precision    recall  f1-score   support

           0       0.99      1.00      0.99     30066
           1       0.96      0.11      0.19       419

    accuracy                           0.99     30485
   macro avg       0.97      0.55      0.59     30485
weighted avg       0.99      0.99      0.98     30485



- **F1 Score** : 정밀도와 재현율의 결합지표
        - 정밀도와 재현율이 모두 중요한 (균형이 필요한 경우에 사용)
        - 0 ~ 1

# Precision & Recall Trade-off

- 비율이 깨진 데이터에 대해서는 Preicision이나 Recall 중 한 쪽이 크면, 다른 한 쪽은 낮은 값으로 계산되는 경향
- Threshold : 분류에 있어서 1을 1로 / 0을 0으로 판단할 때 사용되는 임계값

In [27]:
pd.DataFrame(model.predict_proba(X_test))

,0,1
0,1.000000,0.000000
1,1.000000,0.000000
2,1.000000,0.000000
3,1.000000,0.000000
4,0.983871,0.016129
...,...,...
10157,1.000000,0.000000
10158,0.600000,0.400000
10159,1.000000,0.000000
10160,1.000000,0.000000


- Threshold는 분류된 각 클래스 별 확률 값 중 더 큰 값으로 (0.5/ 50%) 예측을 실시
- Threshold 값을 조정해서, 판단을 다르게 수행
- '해약'으로 판단할 확률이 15% (0.15)만 넘어도 "해약"으로 예측

In [24]:
from sklearn.preprocessing import Binarizer

In [25]:
customer_threshold = 0.15
Binarizer(threshold = customer_threshold) # 모델 분류의 분류기준을 변경

Binarizer(threshold=0.15)

In [26]:
# 해약으로 판단할 확률만 추출
predict_proba_col = model.predict_proba(X_test)[:, 1].reshape(-1,1)

In [28]:
# 새로운 임계값을 적용해 예측
custom_bin = Binarizer(threshold=customer_threshold).fit(predict_proba_col)
Y_test_pred2 = custom_bin.transform(predict_proba_col) #15%를 기준으로 해약을 분류

In [29]:
print(classification_report(Y_test, Y_test_pred2))

              precision    recall  f1-score   support

           0       0.99      0.98      0.98     10038
           1       0.09      0.19      0.12       124

    accuracy                           0.97     10162
   macro avg       0.54      0.58      0.55     10162
weighted avg       0.98      0.97      0.97     10162



- 임계값을 변경하면 실제 F1 score 값이 증가
- 판단 기준을 변경한 것이기에, 모델의 성능은 높아졌다고 볼 수 없음

**Precision과 Recall Curve**
- Threshold 값을 Tunning할 때, Recall과 Precision 값이 가장 높은 지점의 Threshold 값을 확인

In [30]:
from sklearn.metrics import precision_recall_curve

In [31]:
#해약으로 분류할 확률만 추출
Y_test_proba1 = model.predict_proba(X_test)[:, 1]

In [32]:
# 각 Threshold에서 계산된 Recall과 Precision rPtks
precision1, recall1, threshold1 = precision_recall_curve(Y_test, Y_test_proba1)

In [33]:
threshold1

array([0.        , 0.00684932, 0.00714286, 0.00819672, 0.00854701,
       0.01204819, 0.0125    , 0.01265823, 0.01282051, 0.01315789,
       0.01333333, 0.01369863, 0.01428571, 0.01492537, 0.01526718,
       0.01574803, 0.01587302, 0.01612903, 0.01639344, 0.01724138,
       0.01818182, 0.01886792, 0.01960784, 0.02      , 0.02083333,
       0.0212766 , 0.02173913, 0.02222222, 0.02272727, 0.02325581,
       0.02380952, 0.02439024, 0.02469136, 0.025     , 0.02564103,
       0.02608696, 0.02631579, 0.02702703, 0.02739726, 0.02777778,
       0.02857143, 0.02941176, 0.03125   , 0.03225806, 0.03333333,
       0.03448276, 0.03571429, 0.03703704, 0.03846154, 0.04      ,
       0.04166667, 0.04255319, 0.04285714, 0.04347826, 0.04545455,
       0.04761905, 0.05      , 0.05263158, 0.05555556, 0.05882353,
       0.06060606, 0.0625    , 0.06451613, 0.06666667, 0.06818182,
       0.07142857, 0.07407407, 0.07692308, 0.08      , 0.08333333,
       0.08571429, 0.09090909, 0.0952381 , 0.1       , 0.11111

In [34]:
df_cm_curve = pd.DataFrame()
df_cm_curve['threshold'] = threshold1
df_cm_curve['precision'] = precision1[:-1]
df_cm_curve['recall']= recall1[:-1]

In [35]:
# 시각화를 쉽게 하기 위해 데이터 프레임을 재구조화
p1 = df_cm_curve.melt(id_vars=['threshold'])

In [37]:
px.line(p1, x='threshold', y='value', color='variable')

# ROC Curve & AUC

- ROC (Receiver Operation Characteristic Curve)
- '수신자판단곡선'이라고 불림
- 모델에 대한 성능지표를 시각화하여 표현/의학 분야/머신러닝 이진분류 모델
- FPR(False Positive Rate)에 따라 TPR(True Positive Rate, Recall)값이 어떻게 변화하는지를 시각화한 곡선
    - TPR(민감도)와 TNR(특이성 / Specificity) 대한 시각화

In [38]:
from sklearn.metrics import roc_curve

In [39]:
Y_test_proba1 # 해약으로 분류 될 확률값

array([0., 0., 0., ..., 0., 0., 0.])

In [40]:
fprs, tprs, threshold2 = roc_curve(Y_test, Y_test_proba1)

In [41]:
df_roc = pd.DataFrame()
df_roc['FPR'] = fprs
df_roc['TPR'] = tprs
df_roc['Threshold'] = threshold2

In [42]:
import plotly.graph_objects as go # 그래프에 세부 요소를 추가

In [43]:
fig1 = px.line(df_roc, x='FPR', y='TPR')
fig1.add_trace(go.Scatter(x=[0,1], y=[0,1], mode='lines'))

**AUC (Area Under the Curve)**
- ROC에서 아랫 면적을 나타내며, 이 면적은 모델의 성능을 의미
    - AUC가 0.5 : 모델이 데이터를 Random하게 예측 (중앙선에 가깝게 선이 표현)
    - AUC가 1.0 : 모델이 데이터를 완벽하게 예측
    - AUC가 0.7이상인 경우 : 모델이 어느정도 성능을 보임
    - 0.5~0.7 : 모델의 예측이 제한적이며 개선이 필요
    - 실제로는 도메인마다 / 데이터마다 다름!

In [44]:
from sklearn.metrics import roc_auc_score

In [45]:
print('학습 AUC : ', roc_auc_score(Y_train, Y_train_pred))
print('검증 AUC : ', roc_auc_score(Y_test, Y_test_pred))

학습 AUC :  0.5536660238485673
검증 AUC :  0.510552641896278


# 회귀 모델 평가 지표

- R^2 (결정계수) : 변동을 기반으로 예측성능을 평가하는 지표/ 회귀 모델이 데이터를 얼마나 잘 설명하고 있는가 (0 ~ 1)
- MSE (Mean Squared Error) : 실제값과 예측값의 차이를 제곱해 평균을 계산
    - 0으로 갈수록 정확한 모델
    - RMS(Root Mean Square Error)

- MAE (Mean Absolute Error) : 실제값과 예측값의 차이에 절댓값의 평균을 계산
    - 0으로 갈수록 정확한 모델

# Overfitting (과적합)

- 학습 데이터에 대해 성능이 월등히 높게 측정 되나 (100%) 검증 데이터에 대해 성능이 현저히 떨어지는 현상
- 컴퓨터가 학습데이터에 대해 과하게 학습을 수행 -> 일반화
- 해결 : 
    - 학습이 잘 수행되도록 데이터를 깔끔하게 처리
    - 알고리즘을 통제 -> 특성공학(데이터를 처리 + 알고리즘 통제)

## 특성공학 기법을 적용하여 학습

- 특성 공학: 학습을 수행하기 위해 데이터를 깔끔하게 다듬는 과정
- 데이터마이닝에서 알고리즘의 학습 및 일반화 성능을 높이기 위한 기술
- 1. Scaling & Encoding
- 2. Imputation
- 3. Cross Validation
- 4. Hyper Parameter Tuning
- 5. (분류) imbalanced Data Sampling
- 6. Feature Selection
- 7. PCA/ Clustering ...

# 개별미션 (4주차 1일)

In [ ]:
df1 = pd.read_csv('12_Data.csv')

In [ ]:
df1.head(1)

-----------------

1. "Max"이름이 붙은 Column의 기술통계량을 확인

In [ ]:
filtered_columns = df1.filter(like='Max', axis=1)

In [ ]:
filtered_columns.describe()

### 강사님풀이

In [ ]:
col_list = df1.columns.tolist()

In [ ]:
# 리스트 컴프리헨션 사용
df1[[x for x in col_list if 'Max' in x]].describe()

---------

2. Mean Radius(세포 평균 반지름)값에서 평균으로부터, 표준편차의 3배이상 떨어진 데이터의 개수를 확인

In [ ]:
mean2 = df1['Mean Radius'].mean()
std2 = df1['Mean Radius'].std()

In [ ]:
len(df1[df1['Mean Radius'] > (mean2 + std2 * 3)])

### 강사님풀이

In [ ]:
c1 = df1['Mean Radius'] >= df1['Mean Radius'].mean() + (df1['Mean Radius'].std()*3)
c2 = df1['Mean Radius'] <= df1['Mean Radius'].mean() - (df1['Mean Radius'].std()*3)

df1.loc[c1|c2].shape[0]

----------

3. "Diagnosis"(진단 - 암M/정상B 세포 여부)에 따라 Mean Radius의 대표값의 차이가 있는지 가설검정을 수행
    - 가설 및 결론 작성

In [ ]:
df1['Diagnosis'].value_counts()

In [ ]:
cond1 = (df1['Diagnosis'] == 'M')
cond2 = (df1['Diagnosis'] == 'B')

df_M = df1[cond1]
df_B = df1[cond2]

In [ ]:
# 귀무가설 : 암과 정상 진단에 따라 Mean Radius의 대표값에 차이가 없다.
# 대립가설 : 암과 정상 진단에 따라 Mean Radius의 대표값에 차이가 있다.
stats.ranksums(df_M['Mean Radius'], df_B['Mean Radius'])
# P.value < 0.05 /대립가설 참/ 암과 정상 진단에 따라 Mean Radius의 대표값에 차이가 있다.

### 강사님풀이

In [ ]:
# 3-1. 각 집단 별 Mean Radius 값의 정규성 검정부터 실시해야 함
# 귀무가설 : 해당 연속형 데이터의 분포는 정규분포를 따른다.
# 대립가설 : 해당 연속형 데이터의 분포는 정규분포를 따르지 않는다.
print(stats.normaltest(df_M['Mean Radius']))
print(stats.normaltest(df_B['Mean Radius']))

In [ ]:
# 3-2. (정규분포를 따르지 않는/비모수)두 집단의 중앙값을 비교
# 귀무가설 : 두 집단의 해당 연속형 자료의 중앙값은 서로 차이가 없다.
# 대립가설 : 두 집단의 해당 연속형 자료의 중앙값은 서로 차이가 있다.
stats.ranksums(df_M['Mean Radius'],df_B['Mean Radius'])
# P.value<0.05/ 대립가설 참 / 두 집단의 해당 연속형 자료의 중앙값은 서로 차이가 있다.

----------

4. Mean Radius 과 Mean Smoothness(세포 평균 곡률/ 매끄러움 정도)이 서로 상관성이 있는지 가설검정을 수행하고, 상관계수를 확인(Corrleation Coefficant)
    - 가설 및 결론 작성

In [ ]:
# 귀무가설 : Mean Radius와 Mean Smoothness 사이 상관성이 없다.
# 대립가설 : Mean Radius와 Mean Smoothness 사이 상관성이 있다.
stats.spearmanr(df1['Mean Radius'], df1['Mean Smoothness'])
# P.value<0.05/대립가설 참/Mean Radius와 Mean Smoothness 사이 상관성이 있다

In [ ]:
df1[['Mean Radius', 'Mean Smoothness']].corr()

### 강사님풀이

In [ ]:
# 4-1. 두 연속형 자료의 정규성 확인
# 귀무가설 : 해당 연속형 데이터의 분포는 정규분포를 따른다.
# 대립가설 : 해당 연속형 데이터의 분포는 정규분포를 따르지 않는다.
print(stats.normaltest(df_M['Mean Radius']))
print(stats.normaltest(df_B['Mean Radius']))
# P.value<0.05 / 대립가설 참 / 정규성을 따르지 않는다.

In [ ]:
# 4-2. (비모수) 두 연속형 자료의 상관검정
# 귀무가설 : Mean Radius와 Mean Smoothness 사이 상관성이 없다.
# 대립가설 : Mean Radius와 Mean Smoothness 사이 상관성이 있다.
stats.spearmanr(df1['Mean Radius'], df1['Mean Smoothness'])
# P.value<0.05/대립가설 참/Mean Radius와 Mean Smoothness 사이 상관성이 있다

In [ ]:
df1[['Mean Radius', 'Mean Smoothness']].corr(method = 'spearman')

-----------

5. "Mean" 명칭이 포함된 모든 Columns 에 대해 정규성 검정을 실시하고, 정규분포를 따르지 않는 Column을 모두 Histogram을 시각화하는 함수로 구성

In [ ]:
df2 = df1.filter(like='Mean', axis=1)
df2

In [ ]:
# 귀무가설 : 해당 데이터의 분포는 정규분포를 따른다.
# 대립가설 : 해당 데이터의 분포는 정규분포를 따르지 않는다.

def func5():
    df2 = df1.filter(like='Mean', axis=1)
    for i in df2.columns:
        data = df2[i]
        stat, p_value = stats.normaltest(data)
        
        if p_value<0.05:
            fig = px.histogram(df1, x=i)
            fig.show()
            
        else:
            print('정규분포를 따릅니다.')

In [ ]:
func5()

### 강사님 풀이

In [ ]:
def func6():
    col_list_mean = [x for x in df1.columns.tolist() if 'Mean' in x]
    non_normal_list = [] # 정규성을 따르지 않는 항목들을 저장하는 리스트
    for i in col_list_mean:
        if 0.05 >= stats.normaltest(df1[i])[1]:
            non_normal_list.append(i) # 정규성을 따르지 않는 항목들을 추가
            print(i,'항목은 정규분포를 따르지 않는다.')
        else:
            print(i, '항목은 정규분포를 따른다.')
            
    p1 = df1[non_normal_list].melt() # 항목들과 값으로 데이터를 재구조화
    return px.histogram(p1, x='value', color='variable')

In [ ]:
func6()

6. 세포의 기하구조를 입력했을 때, 암/정상 세포를 판단하는 분류모델을 생성
    - 'Mean'명칭이 포함된 모든 Column을 X로 / Diagnosis를 Y로 선언
    - 알고리즘은 DecisionTreeClassifier / Random_state = 1234
    - 생성된 모델을 Model_cell.sav로 저장

In [ ]:
# 목표변수 Y와 설명변수 X 선택
Y = df1['Diagnosis']
X = df1[df2.columns]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# 학습 데이터와 검증 데이터를 분할
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,
                                                   random_state = 1234)

In [ ]:
model = DecisionTreeClassifier()
model.fit(X_train, Y_train)

In [ ]:
import pickle

In [ ]:
pickle.dump(model, open('Model_cell.sav', 'wb'))

----------------

7. 앞서 만든 모델을 아래의 평가 지표를 이용해 평가를 수행하시오.
    - 학습능력과 일반화능력을 모두 평가
    - Acc/ Recall/ Precision/ F1

In [ ]:
# 평가
Y_train_pred = model.predict(X_train)
Y_test_pred = model.predict(X_test)

#학습 능력 평가
print(accuracy_score(Y_train, Y_train_pred))
#일반화 능력 평가
print(accuracy_score(Y_test, Y_test_pred))

In [ ]:
print(classification_report(Y_test, Y_test_pred))

### 강사님풀이

In [ ]:
def evaluation_func1(model):
    Y_train_pred = model.predict(X_train)
    Y_test_pred = model.predict(X_test)
    print('학습능력')
    print(classification_report(Y_train, Y_train_pred))
    print('일반화능력')
    print(classification_report(Y_test, Y_test_pred))

In [ ]:
evaluation_func1(model)

--------------

8. 생성한 분류모델의 Precision과 Recall Curve를 시각화하고, 최적의 Threshold 값을 확인

In [ ]:
Y_test_proba2 = model.predict_proba(X_test)[:, 1]

In [ ]:
precision2, recall2, threshold2 = precision_recall_curve(Y_test, Y_test_proba2)

In [ ]:
threshold2

In [ ]:
df_cm_curve = pd.DataFrame()
df_cm_curve['threshold'] = threshold2
df_cm_curve['precision'] = precision2[:-1]
df_cm_curve['recall']= recall2[:-1]

# 시각화를 쉽게 하기 위해 데이터 프레임을 재구조화
p2 = df_cm_curve.melt(id_vars=['threshold'])

In [ ]:
px.line(p2, x='threshold', y='value', color='variable')

--------------

9. 앞서 계산한 최적의 Threshold 값을 이용해 Test Set을 다시 재분류하고, (M 암세포 기준 -> Positive) 계산된 값을 이용해, AUC값을 확인

In [ ]:
custom_threshold = 0.285
Binarizer(threshold = custom_threshold)

In [ ]:
predict_proba_col2 = model.predict_proba(X_test)[:, 1].reshape(-1,1)

In [ ]:
# 새로운 임계값을 적용해 예측
custom_bin = Binarizer(threshold=customer_threshold).fit(predict_proba_col2)
Y_test_pred2 = custom_bin.transform(predict_proba_col2)

In [ ]:
print(classification_report(Y_test, Y_test_pred2))

In [ ]:
## 강사님 풀아
# 분류 기준값을 86% 재 설정하여, 재 분류한 결과
Binarizer(threshold = 0.86).fit_transform(pred_proba_class2)

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(Y_test, custom_pred)

In [ ]:
from sklearn.metrics import roc_curve

In [ ]:
fpr, tpr, thes = roc_curve(Y_test.replace({'B':0, 'M':1})),pred_proba_class1

In [ ]:
df_roc = pd.DataFrame()
df_roc['FPR'] = fpr
df_roc['TPR'] = tpr

In [ ]:
px.line(df_roc, x='FPR', y='TPR')